In [ ]:
!pip install yahooquery
!pip install pandas
!pip install pandas_ta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 25.9 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=c8e1150a08b610842461b0321770899e9f3be35d973667ddbb574385bc63f6fb
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [ ]:
from prophet import Prophet
from yahooquery import Ticker
import pandas as pd
from datetime import datetime
import pandas_ta as ta
import plotly.express as px
import plotly.graph_objs as go
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics


In [ ]:
# Name by which it is represented on the stock exchange.
symbol ="MXRF11.SA"

# Query stock in yahoo finance
stock = Ticker(symbol)

# Get 48 months data for enough sample terms   /// ORIGINAL (period="48mo")
history = stock.history(period="96mo")

# Let only date as index
history.reset_index(level=["symbol"], inplace=True)

# Create date column
history['date'] = history.index

# Reindex data using a DatetimeIndex
history.set_index(pd.DatetimeIndex(history.index), inplace=True)

# select features that have interest to us
data = history[['date','adjclose']].copy();

# use technical analyses using 21 one days and append to our dataset  //ORIGINAL length=21
data.ta.ema(close='adjclose', length=21, append=True)

# Drop empty values
data.dropna(inplace=True)

In [ ]:
#Plot
fig = px.line(data, x='date', y='adjclose')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

In [ ]:
# Add all dataset as training model
df_train = data[['date','adjclose','EMA_21']]

df_train = df_train.rename(columns={"date": "ds", "adjclose": "y"})

# Fit model
m = Prophet(daily_seasonality=True)

# Train 
m.fit(df_train)

# Get dates 30 days in the future  ////////// valor original: (periods=30)
future = m.make_future_dataframe(periods=730)

# Drop weekends
future['day'] =  future['ds'].dt.weekday
future = future[future['day'] <=4]

# Predict dates
forecast = m.predict(future)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpra2ntnf5/8omxaw91.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpra2ntnf5/zqm_d_ru.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=84918', 'data', 'file=/tmp/tmpra2ntnf5/8omxaw91.json', 'init=/tmp/tmpra2ntnf5/zqm_d_ru.json', 'output', 'file=/tmp/tmpmlpjj3fb/prophet_model-20220914004121.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:41:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:41:22 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
# Plot linear regression result
fig = go.Figure([
    go.Scatter(x=df_train['ds'], y=df_train['y'], name='Actual', mode='lines'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted', mode='lines'),
    go.Scatter(x=forecast['ds'], y=df_train['EMA_21'], name='EMA', mode='lines')
])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
# Get all predictions
pred_df = forecast[forecast['ds'] > datetime.today()][['ds','yhat']]

# Reset Index
pred_df.reset_index(inplace=True)

# Drop index column
pred_df.drop(labels='index', axis=1,inplace=True)

# Rename columns
pred_df.rename(columns={'ds': 'date', 'yhat': 'predicted price'}, inplace=True)

# Show first elements
pred_df

,date,predicted price
0,2022-09-12,7.270989
1,2022-09-13,7.275907
2,2022-09-14,7.280600
3,2022-09-15,7.282828
4,2022-09-16,7.293090
...,...,...
515,2024-09-02,10.151738
516,2024-09-03,10.159249
517,2024-09-04,10.166914
518,2024-09-05,10.172419


In [ ]:
# execute cross validation

# reference
# https://facebook.github.io/prophet/docs/diagnostics.html#:~:text=Cross%20validation,up%20to%20that%20cutoff%20point.

df_cv = cross_validation(m, initial='720 days', period='30 days', horizon = '365 days')

# visualize data
print(df_cv.head())

# measure performance
df_p = performance_metrics(df_cv)
print(df_p.head())

INFO:prophet:Making 61 forecasts with cutoffs between 2016-10-04 00:00:00 and 2021-09-08 00:00:00


  0%|          | 0/61 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp4p9wtzn0/y9s5vvi4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4p9wtzn0/9idn5__w.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=80475', 'data', 'file=/tmp/tmp4p9wtzn0/y9s5vvi4.json', 'init=/tmp/tmp4p9wtzn0/9idn5__w.json', 'output', 'file=/tmp/tmpgmp4i6bt/prophet_model-20220909113607.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:36:07 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:36:08 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4p9wtzn0/ffmcemd1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4p9wtzn0/ff096cja.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-pack

          ds      yhat  yhat_lower  yhat_upper         y     cutoff
0 2016-10-05  2.102567    2.006835    2.196521  1.981658 2016-10-04
1 2016-10-06  2.109934    2.008596    2.203406  1.987487 2016-10-04
2 2016-10-07  2.119275    2.019807    2.208463  1.965339 2016-10-04
3 2016-10-10  2.130708    2.031581    2.223523  1.999726 2016-10-04
4 2016-10-11  2.127838    2.032283    2.224162  2.014881 2016-10-04
  horizon       mse      rmse       mae      mape     mdape     smape  \
0 37 days  0.210579  0.458889  0.334909  0.113019  0.091270  0.113037   
1 38 days  0.215291  0.463994  0.338976  0.114503  0.092130  0.114474   
2 39 days  0.217262  0.466113  0.341418  0.115326  0.093458  0.115288   
3 40 days  0.220525  0.469601  0.344626  0.116485  0.094778  0.116521   
4 41 days  0.224133  0.473427  0.348209  0.117880  0.095889  0.117885   

   coverage  
0  0.366893  
1  0.360947  
2  0.358242  
3  0.353051  
4  0.346773  
